In [1]:
%run stdPackages.ipynb

## 1. Data and settings

In [2]:
t0date = 2010
ngrid = 25
ns0 = 50
t_ss  = 5 # number of periods in steady state
%run ArgentinaData.ipynb

### Simple LOG calibration

In [3]:
m.db.update(m.adjPar('ρ',1))
calsLOG = m.calibLOG()
pathLOG, solsLOG = m.solvePEE()

### CRRA solution:

Start from the LOG solution and solve for CRRA

In [4]:
m.PEE.interpInitialsFromLOG(solsLOG, pathLOG)
m.PEE.kwargs_t['x0_from_solp'] = False # Use x0 instead of solp when iterating over time
m.PEE.kwargsMain = m.PEE.defaultKwargs

Get CRRA solution:

In [5]:
m.db.update(m.adjPar('ρ',1.1))
path, sol = m.solvePEE()

### Test reporting module:

In [6]:
path = m.FH_reportAll(path)

Shock the model and get report:

In [7]:
m.db.update(m.adjPar('eps',m.getEps(coverageRate=1)))
pathNew, solNew = m.solvePEE()

Check out this stuff:

In [8]:
pathNew = m.FH_reportAll(pathNew)

In [9]:
test = m.EV_FH_solve(path, pathNew)

Test stuff:

In [7]:
x = m.x0['EV']
test = m.EV_FH_objective(x, path, path)

In [15]:
test.keys()

dict_keys(['util1i', 'util10', 'util2i', 'util20', 'utilPol'])

In [10]:
self = m
x = m.x0['EV']

In [11]:
syms = {'Δ1i': self.get(x, 'Δ1i', ns = 'EV').unstack('j').values,
        'Δ2i': self.get(x, 'Δ2i', ns = 'EV').unstack('j').values,
        'Δ10': self(x, 'Δ10', ns = 'EV'), 'Δ20': self(x, 'Δ20', ns = 'EV'), 'ΔPol': self(x, 'ΔPol', ns = 'EV')}

In [12]:
sd = path

In [15]:
syms['Δ1i'].shape

(11, 4)

In [24]:
Bip = m.leadSym((1/(1+sd['Bi'])).values)
mpc_1i = 1/(1+Bip)
mpc_2pi = 

In [32]:
syms['Δ1i'] * mpc_1i

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [42]:
sd['R'] = self.BT.R(s_ = sd['s[t-1]'].values, h = sd['h'].values)

In [43]:
(sd['R'][:,None] * Bip/(1+Bip)).shape

(11, 4)

For the young generations, map the 

Before:

In [8]:
# path['Bi'] = pd.DataFrame(m.BT.Bi(s_ = path['s[t-1]'].values, h = path['h'].values), index = m.db['t'], columns = m.db['i'])
# path['Γs[t-1]'] = pd.Series(np.insert(path['Γs'].values, 0, m.B.Γs(Bi = path['Bi'].values[0], τp = path['τ'].values[0], t = m.db['t'][0])), index = m.db['t'])
# pathLOG['Bi'] = pd.DataFrame(m.BT.get('βi'), index = m.db['t'], columns = m.db['i'])
# pathLOG['Γs[t-1]'] = pd.Series(np.insert(path['Γs'].values, 0, m.B.Γs(Bi = path['Bi'].values[0], τp = path['τ'].values[0], t = m.db['t'][0])), index = m.db['t'])
# path['τ[t+1]'] = m.leadSym(path['τ'])
# path['si/s[t-1]'] = pd.DataFrame(m.BT.si_s(Bi = path['Bi'].values, Γs = path['Γs[t-1]'].values, τp = path['τ'].values), index = m.db['t'], columns = m.db['i'])

In [9]:
# path = m.FH_reportCoefficients(path)

In [16]:
pathLOG.keys()

dict_keys(['τ', 'τ[t+1]', 'Γs', 'Θh', 'Θs', 's', 's[t-1]', 'h', 's0/s', 's0/s[t-1]'])

In [39]:
path['Θc2pi']

j,1,2,3,4
t,,,,
0,0.279490,0.430499,0.587690,1.113863
1,0.257143,0.395412,0.539343,1.021126
2,0.242703,0.372846,0.508318,0.961786
3,0.236893,0.363791,0.495884,0.938045
4,0.237075,0.364080,0.496285,0.938820
5,0.237080,0.364087,0.496295,0.938837
6,0.237079,0.364086,0.496293,0.938834
7,0.237082,0.364090,0.496298,0.938844
8,0.237629,0.364925,0.497435,0.940986


In [28]:
path['Θc̃1i'].mul((path['s[t-1]']/m.db['ν'])**m.BT.power_s(), axis = 0)

j,1,2,3,4
t,,,,
0,0.007873,0.012126,0.016554,0.031376
1,0.007930,0.012194,0.016633,0.031491
2,0.007877,0.012101,0.016498,0.031215
3,0.007784,0.011954,0.016295,0.030825
4,0.007702,0.011829,0.016124,0.030501
5,0.007666,0.011773,0.016048,0.030358
6,0.007648,0.011746,0.016011,0.030287
7,0.007640,0.011732,0.015993,0.030253
8,0.007636,0.011726,0.015984,0.030236


In [17]:
path.keys()

dict_keys(['h', 's', 'Γs', 's0/s', 'EE_FH_PEE_x', 's[t-1]', 's0/s[t-1]', 'τ', 'Θh', 'Θs', 'Bi', 'B0', 'τ[t+1]', 'Γs[t-1]', 'si/s[t-1]', 'Θhi', 'Θc̃1i', 'Θc2i', 'Θc2pi', 'Θc̃10', 'Θc20'])

In [12]:
path['s0/s[t-1]']

t
0     0.343421
1     0.349809
2     0.374271
3     0.402769
4     0.421200
5     0.427717
6     0.427374
7     0.427385
8     0.427382
9     0.427444
10    0.443210
dtype: float64

In [13]:
self = m.BT
Θh = path['Θh'].values
Γs = path['Γs'].values
Bi = path['Bi'].values
τp = path['τ[t+1]'].values

In [22]:
path.keys()

dict_keys(['h', 's', 'Γs', 's0/s', 'EE_FH_PEE_x', 's[t-1]', 's0/s[t-1]', 'τ', 'Θh', 'Θs', 'Bi', 'τ[t+1]', 'Γs[t-1]', 'si/s[t-1]', 'Θhi', 'Θc̃1i', 'Θc2i', 'Θc2pi'])

In [24]:
np.vstack([Θ̃c1i_t(self, Θh[:-1], Γs, Bip = Bi[1:,], τp = τp[:-1], t = self.db['txE']),
           Θ̃c1i_T(self, Θh = Θh[-1:], t = self.db['t'][-1:])])

array([[0.10753288, 0.1656329 , 0.22611189, 0.42855503],
       [0.10476621, 0.16110022, 0.2197409 , 0.41603056],
       [0.10001888, 0.15365114, 0.20947943, 0.39635515],
       [0.09670893, 0.14851353, 0.20243934, 0.38294679],
       [0.09523202, 0.14624947, 0.19935589, 0.37712058],
       [0.09530001, 0.14635358, 0.19949761, 0.37738819],
       [0.0952938 , 0.14634398, 0.19948447, 0.37736322],
       [0.09529281, 0.14634241, 0.1994823 , 0.37735903],
       [0.09529459, 0.14634355, 0.19948278, 0.37735728],
       [0.09720066, 0.14894908, 0.20281641, 0.38312811],
       [0.12374916, 0.19172953, 0.26249345, 0.49936359]])

In [ ]:
def Θc2i(self, τ = None, Θh = None, si_s = None, t = None):
	return (self.get('α',t)*self.get('ν',t)*Θh**(1-self.get('α',t))/self.get('p[t-1]',t)) * (si_s + (self.get('αr',t)*self.get('p[t-1]',t)*τ/self.get('κ[t-1]',t))[:,None]*(1+self.get('θ',t)[:,None]*(self.auxProd_(t)/self.get('Γh[t-1]',t)[:,None]-1)))

In [19]:
def Θ̃c1i_t(self, Θh = None, Γs = None, Bip = None, τp = None, t = None):
    return ((Θh/self.get('Γh',t))**((1+self.get('ξ',t))/self.get('ξ',t)))[:,None]*(self.auxProd(t)/(1+self.get('ξ',t)[:,None])+(Γs*self.get('αr',t)* self.get('p',t)*τp*(1-self.get('θ[t+1]',t))/self.get('κ',t))[:,None])/(1+Bip)
def Θ̃c1i_T(self, Θh = None, t = None):
	return ((Θh/self.get('Γh',t))**((1+self.get('ξ',t))/self.get('ξ',t))/(1+self.get('ξ',t)))[:,None] * self.auxProd(t)